# Spam Filtering with Naive Bayes
In this project, we will be constructing a spam filtering program that uses multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages which has been classified by humans.  

When looking at the steps a computer will take to classify incoming new messages as spam, the steps taken are pretty simiple.  They consist of:

- Learning how humans classify messages

- Use the human knowledge to estimate probabilities for new messages (spam vs non-spam)

- Based on which has a higher probability (spam vs non-spam), the computer will classify the new message as spam or non-spam.

Using the dataset put together by Tiago A. Almeida and Jose Maria Gomez Hidalgo, we'll help the computer learn to classify spam vs non spam.  We'll need to split the data into training and testing sets.  This way after we train the algorithm on the training set, we can then run the model on the test set to predit the classifications.  Hopefully, if the model is succesful, using the simple Naive Bayes algorithm, we'll be able to classify incoming messages with high accuracy.  Ideally we'd like to exceed 80%.

### Import the Data
We'll first want to import the data and get familiarized with it.  The data is tab separated and has no headers, so we'll need to update the read parameters to account for this.

In [1]:
import pandas as pd
sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names = ['Label','SMS'])


In [2]:
sms.shape

(5572, 2)

In [3]:
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
sms['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

The data looks organized correctly and has about 86% regular messages.  This seems pretty accurate as most e-mails people get will be non-spam.

### Train and Test Sets
In order for our algorithm to gain higher accuracy, we'll want to train the algorithm on part of the data and then test the algorithm on a different part.  This way as the alogorithm learns, we can test the accuracy.  For this dataset, we'll split the data as following:

- Training set will have 4,458 messages (80%)

- Test set will have 1,114 messages (20%)

We'll want to first randomize all the data and then use the <code>random_state</code> parameter to make sure our results our reproducible.

In [5]:
#randomize the data
random_sms = sms.sample(frac=1,random_state=1)

#80/20 split index
training_split = round(len(random_sms) *.80)

#split the data
train_sms = random_sms[:training_split].reset_index(drop=True)
test_sms = random_sms[training_split:].reset_index(drop=True)

print('Train: ',train_sms.shape)
print('Test: ',test_sms.shape)

Train:  (4458, 2)
Test:  (1114, 2)


In [6]:
train_sms['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test_sms['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Both the test and training set have approximately the same breakdown of spam vs non spam.  They also both match the original dataset breakdown.  That means we're good to go!

### Data Cleaning
Now that we have our Training and Test sets, before we beging caluclating probabilities and applying the Naive Bayes algorithm, we'll first need to clean the data.

We'll want to make sure our message vocabulary is consistent, so we'll bring everything down to lowercase and we'll remove punctuation.  

Ultimately, we'll want to have each row contain counters of the words used in each message in our training set.  For example, the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", will be converted to counters.  We want our new data set to look like the following:

| Label | secret | prize | claim | now | coming | to | my | party | winner |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| spam | 2 | 2 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| ham | 1 | 0 | 0 | 0 | 1 | 1 | 1 | 1 | 0 |
| spam | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 1 |

Let's begin by removing punctuation and lowercasing the training set.

In [8]:
train_sms['SMS'] = train_sms['SMS'].str.replace('\W',' ').str.lower()
train_sms.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating a vocabulary list
Now that we have the SMS messages cleaned, we need to put together a master list of each word used.  We'll iterate over each word and add it to a list.  We'll then transform the list into a set which will remove duplicates and then bring it back to a list.

In [9]:
#split the sms column
train_sms['SMS'] = train_sms['SMS'].str.split()
vocab_list = []
for message in train_sms['SMS']:
    for word in message:
        vocab_list.append(word)

vocab = list(set(vocab_list))
print('Total vocabulary words: ',len(vocab))

Total vocabulary words:  7783


We can see that there are a total of 7783 unique words used in all of the training SMS messages.

### Final Training Set
Now that we have the vocabulary list we can now create our final training dataset that will have vocabulary counts for each message.

In [10]:
#Create the unique word dictionary
word_counts_per_sms = {unique_word: [0] * len(train_sms['SMS']) for 
                      unique_word in vocab}
#Count each word
for index,sms in enumerate(train_sms['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,deluxe,hugs,hits,pillows,ls15hb,failing,hrs,smidgin,careless,visa,...,wotu,gautham,afraid,447801259231,okey,inconsiderate,trivia,pretend,do,leftovers
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that we have a dataset made with the vocab list count, we can combine it with our original training set.

In [11]:
train_sms_clean = pd.concat([train_sms,word_counts],axis=1)
train_sms_clean.head()

,Label,SMS,deluxe,hugs,hits,pillows,ls15hb,failing,hrs,smidgin,...,wotu,gautham,afraid,447801259231,okey,inconsiderate,trivia,pretend,do,leftovers
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating the Constants
Now that we have the dataset clean, we'll have to use the following probability equations to create our spam filter.

These are the equations we'll need to solve for to properly implement the Naive Bayes Algorithm:

-----------------------

$$P(Spam|w_1,w_2,...,w_n) = P(Spam)\cdot\prod_{i=1}^{n}P(w_i|Spam)$$ 

$$P(Non Spam|w_1,w_2,...,w_n) = P(Non Spam)\cdot\prod_{i=1}^{n}P(w_i|NonSpam)$$ 

-----------------------

$$P(w_i|Spam) = \frac{(N_{wi|Spam} + \alpha)}{(N_{Spam} + {\alpha}\cdot{N_{Vocab}})}$$

$$P(w_i|Non Spam) = \frac{(N_{wi|NonSpam} + \alpha)}{(N_{Non Spam} + {\alpha}\cdot{N_{Vocab}})}$$

-----------------------

First, we'll need to calculate the constants for those equations.  So we'll begin by calculating the following:
- $P(Spam)$ and $P(NonSpam)$
- $N_{spam}, N_{nonSpam}, N_{vocab}$

We need to keep in mind that the following holds true:
- $N_{spam}$ is equal to the number of words in all spam messages (not equal to number of spam messages and not equal to total number of unique words in spam messages).

- $N_{NonSpam}$ is equal to the number of words in all non-spam messages (not equal to number of non-spam messages and not equal to total number of unique words in non-spam messages).

In [12]:
#filter based on label
spam_sms = train_sms_clean[train_sms_clean['Label'] == 'spam']
non_spam_sms = train_sms_clean[train_sms_clean['Label'] == 'ham']

#basic probability of spam vs non spam
p_spam = len(spam_sms) / len(train_sms_clean)
p_non_spam = len(non_spam_sms) / len(train_sms_clean)

#calculate all words used in spam messages
n_spam = spam_sms['SMS'].apply(len).sum()

#calculate all words used in non spam messages
n_non_spam = non_spam_sms['SMS'].apply(len).sum()

#calculate all words used in vocabulary
n_vocab = len(vocab)

#set alpha parameter for Laplace smoothing to 1
alpha = 1

### Calculating all parameters
Now that we have the constants set, we'll want to calculate the parameters of each word is classified as spam or not spam.

We'll do this by first calculating the parameters of both with the following equations.

$$P(w_i|Spam) = \frac{(N_{wi|Spam} + \alpha)}{(N_{Spam} + {\alpha}\cdot{N_{Vocab}})}$$

$$P(w_i|Non Spam) = \frac{(N_{wi|NonSpam} + \alpha)}{(N_{Non Spam} + {\alpha}\cdot{N_{Vocab}})}$$

Calculating all of the parameters beforehand will help speed up our classifier when testing and in a real world situation.  Rather than having each calculation done when a new message arrives, it will already be done. 

In [13]:
#initialize both dictionaries
spam_parameters = {unique_word:0 for unique_word in vocab}
non_spam_parameters = {unique_word:0 for unique_word in vocab}

#calculate probabilities
for word in vocab:
    n_word_given_spam = spam_sms[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocab)
    spam_parameters[word] = p_word_given_spam
    
    n_word_given_non_spam = non_spam_sms[word].sum()
    p_word_given_non_spam = (n_word_given_non_spam + alpha) / (n_non_spam + alpha*n_vocab)
    non_spam_parameters[word] = p_word_given_non_spam
    

### Classifiying a new message
Now that we have all of the parameters calculated, we can now begin creating our spam filter.  We'll want our function to do the following:

- Takes in as input a new message ($w_1, w_2, ..., w_n$).
- Calculates $P(Spam|w_1, w_2, ..., w_n)$ and $P(Non Spam|w_1, w_2, ..., w_n)$.
- Compares the values of $P(Spam|w_1, w_2, ..., w_n)$ and $P(Non Spam|w_1, w_2, ..., w_n)$, and:
    - If $P(Non Spam|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as non spam.
    - If $P(Non Spam|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n)$, then the message is classified as spam.
    - If $P(Non Spam|w_1, w_2, ..., w_n) = P(Spam|w_1, w_2, ..., w_n)$, then the algorithm may request human help.
    
Here is where we will implment the Naive Bayes Algorithm.  Again, it is as follows:

$$P(Spam|w_1,w_2,...,w_n) = P(Spam)\cdot\prod_{i=1}^{n}P(w_i|Spam)$$ 

$$P(Non Spam|w_1,w_2,...,w_n) = P(Non Spam)\cdot\prod_{i=1}^{n}P(w_i|NonSpam)$$ 

In [14]:
#Define classify function
import re
def classify(message):
    message = re.sub('\W',' ',message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_non_spam_given_message = p_non_spam
    
    for word in message:  
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]
        
        if word in non_spam_parameters:
            p_non_spam_given_message *= non_spam_parameters[word]
    
    print('P(Spam|message): ',p_spam_given_message)
    print('P(Non_Spam|message): ',p_non_spam_given_message)
    
    if p_non_spam_given_message > p_spam_given_message:
        print('Label: Non Spam')
    elif p_non_spam_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal probabilities, have a human classify this!')

In [15]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message):  1.3481290211300841e-25
P(Non_Spam|message):  1.9368049028589875e-27
Label: Spam


In [16]:
classify('Sounds good, Tom, then see u there')

P(Spam|message):  2.4372375665888117e-25
P(Non_Spam|message):  3.687530435009238e-21
Label: Non Spam


So far it looks good.  Since we tuned our algorithm based on the training set, we can now test it on the test set to see how well our spam filter does at prediciting correctly. 

### Measuring Accuracy on the test set
Now we can update the classifier function a bit to return labels so we can create a new column in our test set that displays the prediction.

In [17]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_non_spam_given_message = p_non_spam

    for word in message:
        if word in spam_parameters:
            p_spam_given_message *= spam_parameters[word]

        if word in non_spam_parameters:
            p_non_spam_given_message *= non_spam_parameters[word]

    if p_non_spam_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_non_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [18]:
test_sms['predicted'] = test_sms['SMS'].apply(classify_test_set)
test_sms.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Briefly looking at the first 5, the predictions look like they are accurate.  Let's test the accuracy.

In [19]:
correct = 0
total = test_sms.shape[0]

for row in test_sms.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
print('Correct: ', correct)
print('Incorrect: ',total-correct)
print('Accuracy: ',correct/total)

Correct:  1100
Incorrect:  14
Accuracy:  0.9874326750448833


It looks like our spam filter worked extremely well.  It correctly classified 98.7% of all e-mails!

### Increasing Accuracy
Given that our model is pretty accurate, we could leave it as is, or we could investigate further to see if we can tweak it a bit to increase accuracy.  We can begin by looking at the misclassified messages.

In [20]:
test_sms[test_sms['Label'] != test_sms['predicted']]

,Label,SMS,predicted
114,spam,Not heard from U4 a while. Call me now am here...,ham
135,spam,More people are dogging in your area now. Call...,ham
152,ham,Unlimited texts. Limited minutes.,spam
159,ham,26th OF JULY,spam
284,ham,Nokia phone is lovly..,spam
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,needs human classification
302,ham,No calls..messages..missed calls,spam
319,ham,We have sent JD for Customer Service cum Accou...,spam
504,spam,Oh my god! I've found your number again! I'm s...,ham
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham


It seems that potentially words not in the vocabulary (ie. 146tf150p), general context of the message, or referencing phone related topics cause misclassifcations.  If we want, we can make our model a bit more robust by accounting for this.

# Conclusion
Using a relatively simple algorithm, the Naive Bayes algorithm, we were able to build a very functional spam filtering program that was able to classify message with almost a 99% accuracy rate.